In [45]:
# imports

import pandas as pd
import requests
from dotenv import load_dotenv
import os
import numpy as np
import json




# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [19]:
# Access API Keys
load_dotenv(dotenv_path='/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data')
api_key = os.getenv('FOURSQUARE_Project_API_KEY')

In [23]:
# Upload Lisbon CityBike Data .CSV
lb_citybikes = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data/lisbon_bike_station_df.csv')

lb_citybikes

,latitude,longitude,timestamp,free bikes,empty slots,total slots
0,38.753590,-9.157170,2025-07-25T01:06:43.220041+00:00Z,1,31,32
1,38.752310,-9.158850,2025-07-25T01:06:43.219966+00:00Z,0,23,23
2,38.770657,-9.160248,2025-07-25T01:06:43.219971+00:00Z,5,28,33
3,38.780690,-9.096220,2025-07-25T01:06:43.220291+00:00Z,10,4,14
4,38.702936,-9.175234,2025-07-25T01:06:43.220528+00:00Z,8,15,23
...,...,...,...,...,...,...
190,38.729724,-9.157531,2025-07-25T01:06:43.220410+00:00Z,4,26,30
191,38.711498,-9.194361,2025-07-25T01:06:43.220763+00:00Z,2,39,41
192,38.739395,-9.162447,2025-07-25T01:06:43.220169+00:00Z,0,17,17
193,38.708780,-9.137125,2025-07-25T01:06:43.220502+00:00Z,13,7,20


In [27]:
# Parameters
lb_citybikes['coordinates'] = lb_citybikes['latitude'].astype(str) + ',' + lb_citybikes['longitude'].astype(str)

lb_citybikes.head(5)

,latitude,longitude,timestamp,free bikes,empty slots,total slots,coordinates
0,38.753590,-9.157170,2025-07-25T01:06:43.220041+00:00Z,1,31,32,"38.75359,-9.15717"
1,38.752310,-9.158850,2025-07-25T01:06:43.219966+00:00Z,0,23,23,"38.75231,-9.15885"
2,38.770657,-9.160248,2025-07-25T01:06:43.219971+00:00Z,5,28,33,"38.770657,-9.160248"
3,38.780690,-9.096220,2025-07-25T01:06:43.220291+00:00Z,10,4,14,"38.78069,-9.09622"
4,38.702936,-9.175234,2025-07-25T01:06:43.220528+00:00Z,8,15,23,"38.702936,-9.175234"


In [46]:
# Create Parameters
ll = lb_citybikes['coordinates'].head(10)

# Assign Parameters
paramters = {
    'll' : ll,
    'radius':1000,
    'fields':'name,fsq_place_id,distance,categories,attributes,price,popularity,rating,stats,hours,hours_popular,veracity_rating',
}

url = 'https://places-api.foursquare.com/places/search'

headers = {
    'accept': 'application/json',
    'authorization': 'Bearer'+api_key,
    'X-Places-Api-Version': '2025-06-17'
}

ll


0      38.75359,-9.15717
1      38.75231,-9.15885
2    38.770657,-9.160248
3      38.78069,-9.09622
4    38.702936,-9.175234
5      38.73812,-9.14617
6      38.75115,-9.15907
7      38.73617,-9.13672
8    38.744521,-9.097661
9    38.731424,-9.135356
Name: coordinates, dtype: object

In [47]:
# FOURSQUARE Places API

for coordinates in ll:

    paramters = {
    'll' : ll,
    'radius':1000,
    'fields':'name,fsq_place_id,distance,categories,attributes,price,popularity,rating,stats,hours,hours_popular,veracity_rating',
    }
    
    response = requests.get(url,headers=headers,params=paramters)
    data = response.json()
    data

In [53]:
import pprint

pprint.pprint(data['results'][0])
# data['context']

{'attributes': {},
 'categories': [{'fsq_category_id': '4bf58dd8d48988d126941735',
                 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
                          'suffix': '.png'},
                 'name': 'Government Building',
                 'plural_name': 'Government Buildings',
                 'short_name': 'Government'},
                {'fsq_category_id': '4bf58dd8d48988d12f941735',
                 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
                          'suffix': '.png'},
                 'name': 'Library',
                 'plural_name': 'Libraries',
                 'short_name': 'Library'}],
 'distance': 124,
 'fsq_place_id': '4cf949b7feec6dcb1d0f3836',
 'hours': {'is_local_holiday': False},
 'hours_popular': [{'close': '2000', 'day': 1, 'open': '0800'},
                   {'close': '2300', 'day': 1, 'open': '2200'},
                   {'close': '1900', 'day': 2, 'open': '0900'},

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [1]:
# Yelp](https://docs.developer.yelp.com/docs/fusion-intro) 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating